# Import

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark_session = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql.functions import *

# Read Data

In [ ]:
items = spark_session.read.option("inferSchema","true").csv("items.csv", header=True, sep="|")

In [ ]:
# items.show(3)

# Data Preprocessing

In [ ]:
# items.schema

In [ ]:
items = items.withColumn("subtopics",translate(items["subtopics"],"[",""))
items = items.withColumn("subtopics",translate(items["subtopics"],"]",""))

### Delete Item 62676

In [ ]:
# items.select([count(when(isnull('main topic'), True))]).show()
# items.select([count(when(isnull('subtopics'), True))]).show()
# items.select([count(when(isnull('subtopics') & isnull('main topic'), True))]).show()
# items.select([count(when(items["subtopics"] == "", True))]).show()

In [ ]:
# items.where((isnull('subtopics') & isnull('main topic'))).show()

In [ ]:
# items.where("itemID = 62676").show()

In [ ]:
items = items.filter(items.itemID != 62676)

In [ ]:
# items.where((isnull('subtopics') & isnull('main topic'))).show()

In [ ]:
# items.show(3)

### Combine Main Topic and Subtopics

In [ ]:
items = items.withColumn(
    "topics", 
    when(isnull("main topic"), items["subtopics"]).
    when(items["subtopics"] == "", items["main topic"]).
    otherwise(concat(col("main topic"), lit(","), col("subtopics")))
) 

In [ ]:
# items.show(3)
# items.where(isnull('main topic')).show(5)
# items.where(items["subtopics"] == "").show(3)

In [ ]:
items = items.select("itemID","title","author","publisher","topics")

In [ ]:
# items.show(3)

In [ ]:
items = items.withColumn(
    'topics', array_distinct(split(col("topics"),","))
)

In [ ]:
# items.select("topics").show(5, False)
# items.show(3)

### Explode into separate rows

In [ ]:
from pyspark.sql.functions import explode

In [ ]:
items = items.withColumn("topics_splitted", explode(items.topics))

In [ ]:
# items.show(5)

In [ ]:
items = items.drop("topics")

In [ ]:
# items.show(3)

### Pivot

In [ ]:
from pyspark.sql.functions import sum

In [ ]:
spark_session.conf.set("spark.sql.pivotMaxValues",25000)

In [ ]:
pivot = items.groupBy("itemID").pivot("topics_splitted").count()

In [ ]:
pivot = pivot.fillna(0)

In [ ]:
# pivot.show()

### Dimensionality Reduction

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

In [ ]:
mat = pivot.drop("itemID").rdd.map(lambda s : Vectors.dense(s))

In [ ]:
mat = RowMatrix(mat)

### Principal component analysis (PCA)

In [ ]:
pca = mat.computePrincipalComponents(5)

In [ ]:
projected = mat.multiply(pca)

In [ ]:
# print(projected.rows.collect())